# Connect RDBS using jdbc connector from EMR Studio Notebook using Pyspark, Spark Scala, and SparkR

#### Topics covered in this example

* Configuring jdbc driver
* Connecting to database using jdbc to read data
* Connecting to database using jdbc to write data

## Table of Contents:

1. [Prerequisites](#Prerequisites)
2. [Introduction](#Introduction)
3. [Upload the MySQL jdbc driver in S3 and declare the path](#Upload-the-MySQL-jdbc-driver-in-S3-and-declare-the-path)
4. [Read and write data using Pyspark](#Read-and-write-data-using-Pyspark)
5. [Read and write data using Scala](#Read-and-write-data-using-Scala)
6. [Read and write data using SparkR](#Read-and-write-data-using-SparkR)


## Prerequisites

Download jdbc driver and upload it on S3 which is accessible from the Amazon EMR cluster attached to the Amazon EMR Studio. 

## Introduction

This notebooks shows how to connect RDBS using jdbc connector from Amazon EMR Studio Notebook. 

## Upload the MySQL jdbc driver in S3 and declare the path

In [ ]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://<S3 Bucket>/jars/mysql-connector-java-8.0.19.jar"        
    }
}

## Read and write data using Pyspark

In [ ]:
%%pyspark

#Declare the variables and replace the variables values as appropiate

str_jdbc_url='jdbc:mysql://<Database endpoint>:3306/<Database name>'
str_Query= '<Select query>'
str_dbname='<Database name>'
str_username='<Username>'
str_password='<Password>'
str_tgt_table='<Target Table>'

# Read data from source table

jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", str_jdbc_url) \
    .option("query", str_Query) \
    .option("user", str_username) \
    .option("password", str_password) \
    .load()

jdbcDF.limit(5).show()

# Write data to the target database

jdbcDF.write \
    .format("jdbc") \
    .option("url", str_jdbc_url) \
    .option("dbtable", str_tgt_table) \
    .option("user", str_username) \
    .option("password", str_password).mode('append').save()


## Read and write data using Scala

In [ ]:
%%scalaspark

#Declare the variables and replace the variables values as appropiate

val str_jdbc_url="jdbc:mysql://<Database endpoint>:3306/<Database name>"
val str_Query= "<Select Query>"
val str_dbname="<Database name>"
val str_username="<Username>"
val str_password="<Password>"
val str_tgt_table="<Target table>"

# Read data from source table

val jdbcDF = (spark.read.format("jdbc")
  .option("url", str_jdbc_url)
  .option("query", str_Query)
  .option("user", str_username)
  .option("password", str_password)
  .load())

jdbcDF.limit(5).show()

# Write data to the target database

val connectionProperties = new java.util.Properties
connectionProperties.put("user", str_username)
connectionProperties.put("password", str_password)

jdbcDF.write.mode("append").jdbc(str_jdbc_url, str_tgt_table, connectionProperties)


## Read and write data using SparkR

In [ ]:
%%rspark

#Declare the variables and replace the variables values as appropiate

str_jdbc_url="jdbc:mysql://<Database endpoint>:3306/<Database name>"
str_dbname="<Database name>"
str_username="<Username>"
str_password="<Password>"
str_tgt_table="<Target table>"

# Read data from source database

df <- read.jdbc(str_jdbc_url, 
                "(select employee_id, first_name, last_name, email, dept_name from notebook.employee e, notebook.dept d where e.department_id = d.department_id) AS tmp", 
                user = str_username, 
                password = str_password)

showDF(df)

jdbcDF.limit(5).show()

# Write data to the target database

write.jdbc(df, 
           str_jdbc_url, 
           str_tgt_table,
           user = str_username,
           password = str_password,
           mode = "append")
